#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2020


# Homework 3:   Recommender System Practice: Rating Prediction and Top-K Item Recommendation

### 100 points [ 6% of your final grade]

### Due: April 10, 2020

*Goals of this homework:* Understand matrix factorization (MF) using explicit feedback and Bayesian Personalized Ranking (BPR) using implicit feedback for recommendation. Explore different methods for two real-world recommendation senarios: rating prediction and top-K item recommendation.

*Submission instructions (eCampus):* To submit your homework, rename this notebook as `UIN_hw3.ipynb`. For example, my homework submission would be something like `555001234_hw3.ipynb`. Submit this notebook via eCampus (look for the homework 3 assignment there). Your notebook should be completely self-contained, with the results visible in the notebook. We should not have to run any code from the command line, nor should we have to run your code within the notebook (though we reserve the right to do so). So please run all the cells for us, and then submit.

*Late submission policy:* For this homework, you may use as many late days as you like (up to the total late days you have remaining).

*Collaboration policy:* You are expected to complete each homework independently. Your solution should be written by you without the direct aid or help of anyone else. However, we believe that collaboration and team work are important for facilitating learning, so we encourage you to discuss problems and general problem approaches (but not actual solutions) with your classmates. You may post on Piazza, search StackOverflow, etc. But if you do get help in this way, you must inform us by **filling out the Collaboration Declarations at the bottom of this notebook**. 

*Example: I found helpful code on stackoverflow at https://stackoverflow.com/questions/11764539/writing-fizzbuzz that helped me solve Problem 2.*

The basic rule is that no student should explicitly share a solution with another student (and thereby circumvent the basic learning process), but it is okay to share general approaches, directions, and so on. If you feel like you have an issue that needs clarification, feel free to contact either me or the TA.

# Part 1. Matrix Factorization for Rating Prediction (70 points total)

In some platforms, such as MovieLens, users express their preference on items using explict feedback like ratings.

In this part, you will implement matrix factorization to predict ratings on MovieLens data. After removing users who left less than 20 ratings and movies with less than 20 ratings, the provided dataset has only ~1,200 items and ~500 users. You can also check the title and genres of each movie in *movies_info.csv*.

## Part 1a: Load the Data (5 points)

Please download the dataset from Piazza. There are about 65,000 ratings in total. We split the rating data into two sets. You will train with 70% of the data (in *train_movie.csv*) and test on the remaining 30% of data (in *test_movie.csv*). Each of train and test files has lines having this format: UserID, MovieID, Rating. 

First you will need to load the data and store it with any structure you like. Please report the numbers of unique users and movies in the dataset. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import math
from functools import reduce

import surprise
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate


In [2]:
# load the data, then print out the number of
# movies and users in each of train and test sets.
# Your Code Here...

trainset = pd.read_csv('train_movie.csv')
print("         TRAIN DATA ")
print("==========================")
print(trainset.head())
print()
print("         TEST DATA ")
print("==========================")
testset = pd.read_csv('test_movie.csv')
print(testset.head())

unique_users_trainset =  len(trainset.userId.unique())
unique_movies_trainset = len(trainset.movieId.unique())

unique_users_testset =  len(testset.userId.unique())
unique_movies_testset = len(testset.movieId.unique())

print()
print('Number of unique users in Train Data:', unique_users_trainset )
print('Number of unique movies in Train Data:', unique_movies_trainset )
print()
print('Number of unique users in Test Data :', unique_users_testset )
print('Number of unique movies in Test Data :', unique_movies_testset)

         TRAIN DATA 
   userId  movieId  rating
0       0        4       4
1       0       23       5
2       0       25       5
3       0       31       3
4       0       33       5

         TEST DATA 
   userId  movieId  rating
0       0      635       4
1       0      587       5
2       0      339       5
3       0      250       5
4       0      411       5

Number of unique users in Train Data: 541
Number of unique movies in Train Data: 1211

Number of unique users in Test Data : 541
Number of unique movies in Test Data : 1211


## Part 1b: Matrix Factorization (40 points)

In class, we introduced how matrix factorization works for recommendation. Now it is your term to implement it. There are different methods to obtain the latent factor matrices **P** and **Q**, like gradient descent, Alternating Least Squares (ALS), and so on. Pick one of them and implement your MF model. *You can refer to tutorials and resources online. Remember our **collaboration policy** and you need to inform us of the resources you refer to.* 

Please report MAE and RMSE of your MF model for the test set.

In [5]:
# Your Code Here...
# Report Mean Absolute Error and Root Mean Squared Error for test


In [3]:
rating_matrix = pd.pivot_table(trainset,values='rating',index=['movieId'],columns=['userId'],fill_value=0)

In [4]:
K = 2
R = np.array(rating_matrix)
N = len(R)  # 1211
M = len(R[0]) #541

U = np.random.rand(N,K)
V = np.random.rand(M,K)
iterations = 80
alpha = 0.0002

In [5]:
def MF_Gradient_Descent(R,U,V,K,iterations,alpha):
    V = V.T 
    for it in range(iterations):  
        for i in range(len(R)): 
            for j in range(len(R[i])):  
                if R[i][j] > 0:
                    err = R[i][j] - np.dot(U[i,:],V[:,j])  #find error
                    
                    for k in range(K):
                        U[i][k] = U[i][k] + alpha * (2 * err * V[k][j])
                        V[k][j] = V[k][j] + alpha * (2 * err * U[i][k])
        R1 = np.dot(U,V)
        error = 0
        for x in range(len(R)):
            for y in range(len(R[x])):
                if R[x][y] > 0:
                    error = error + pow(R[x][y] - np.dot(U[x,:],V[:,y]), 2)
                    for z in range(K):
                        error = error + ( pow(U[x][z],2) + pow(V[z][y],2) )
        if error < 0.001:
            break                
    return U, V.T                    

In [6]:
Ui , Vi = MF_Gradient_Descent(R,U, V, K,iterations,alpha)
predicted_matrix = np.dot(Ui,Vi.T)

In [7]:
def predictions(testset):
    predicted_results, actual_results = [], []
    for col,row in testset.iterrows():
        actual_results.append(row['rating'])
        predicted_results.append(predicted_matrix[row['movieId']][row['userId']])
    return actual_results,predicted_results

In [8]:
#CALCULATION OF RMSE AND MAE OF MODEL FOR TEST SET
def calculate_RMSE(original_rating, predicted_rating):
    err = 0
    total_ratings = len(original_rating)
    for i in range(0,total_ratings):
        err = err + math.pow((original_rating[i] - predicted_rating[i]),2)
    err = err / len(original_rating)
    rmse = math.sqrt(err)
    return rmse

def calculate_MAE(original_rating, predicted_rating):
    err = 0
    total_ratings = len(original_rating)
    for i in range(0,total_ratings):
        err = err + abs((original_rating[i] - predicted_rating[i]))
    mae = err /  total_ratings    
    return mae

In [19]:
MF_actual_ratings,MF_predicted_ratings = predictions(testset)

print('  Matrix Factorization Using Gradient Descent')
print("==================================================")
print('The Mean Absolute Error for Test Data is :', calculate_MAE(MF_actual_ratings,MF_predicted_ratings))
print('The Root Mean Squared Error for Test Data is :', calculate_RMSE(MF_actual_ratings,MF_predicted_ratings))


  Matrix Factorization Using Gradient Descent
The Mean Absolute Error for Test Data is : 0.6940211809384695
The Root Mean Squared Error for Test Data is : 0.9027726341425492


Which method did you use to obtain **P** and **Q**? What are the advantages and disadvantages of the method you pick? *provide a brief (1-2 paragraph) discussion based on these questions.*

**Gradient Descent training algorithm was used to compute P and Q (latent features) which are Ui and Vi in my code.It is a convex optimisation method. How far we are at one time (prediction) tells us how to improve prediction and which direction to go.This is done by intializing the two matrices with some values, calculate how different their product is to R (finding error) and then try to minimize this difference iteratively.This method aims at finding a local minimum of the difference**

**Advantages include its cost effectiveness, computational efficency and that it produces a stable error gradient and a stable convergence. It is computationally fast because it doesnt require any computation of second-derivatives i.e only one sample is processed at a time thus making it give speedier results.It is easier to fit into memory due to a single training sample being processed by the network. For larger datasets it can converge faster as it causes updates to the parameters more frequently.**

**Some disadvantages are that the stable error gradient can sometimes result in a state of convergence that isn't the best the model can achieve.The learning rate can affect which minimum we reach and how quickly we reach it.If the learning rate for gradient descent is too fast, we are going to skip the true local minimum to optimize for time. If it is too slow, the gradient descent may never converge because it is trying really hard to exactly find a local minimum.**

## Part 1c: Improve MF (25 points)

Given your results in the previous part, can you do better? For this last part you should report on your best attempt at improving MAE and RMSE. Provide code, results, plus a brief discussion on your approach. Hints: You may consider using the title or genres information, trying other algorithms, designing a hybrid system or considering a neighborhood like this paper [Factorization Meets the Neighborhood: a Multifaceted Collaborative Filtering Model](https://www.cs.rochester.edu/twiki/pub/Main/HarpSeminar/Factorization_Meets_the_Neighborhood-_a_Multifaceted_Collaborative_Filtering_Model.pdf). *You can do anything you like to improve MAE and RMSE.*

You will get full marks for this part if you get better results than your MF results (of course we will also judge whether what you do here is reasonable or not). You will get partial marks for a reasonable effort even if you do not improve your MF results. Additionally, you will get 5 points as bonus if your model performs the best among the whole class.

In [10]:
regularisation_term  = 0.09

In [11]:
def MF_Gradient_Descent_Improvement(R,U,V,K,iterations,alpha,beta):
    V = V.T 
    for it in range(iterations):  
        for i in range(len(R)):  
            for j in range(len(R[i])): 
                if R[i][j] > 0:
                    err = R[i][j] - np.dot(U[i,:],V[:,j])  
                    
                    for k in range(K):
                        U[i][k] = U[i][k] + alpha * (2 * err * V[k][j] - beta * U[i][k])
                        V[k][j] = V[k][j] + alpha * (2 * err * U[i][k] - beta * V[k][j])
        R1 = np.dot(U,V)
        error = 0
        for x in range(len(R)):
            for y in range(len(R[x])):
                if R[x][y] > 0:
                    error = error + pow(R[x][y] - np.dot(U[x,:],V[:,y]), 2)
                    for z in range(K):
                        error = error + (beta/2) * ( pow(U[x][z],2) + pow(V[z][y],2) )
        if error < 0.001:
            break                
    return U, V.T                    

In [12]:
P, Q = MF_Gradient_Descent_Improvement(R,U, V, K,iterations,alpha,regularisation_term)

In [13]:
predicted_matrix_impr = np.dot(P,Q.T)

In [14]:
def predictions_imp(testset):
    predicted_results, actual_results = [], []
    for col,row in testset.iterrows():
        actual_results.append(row['rating'])
        predicted_results.append(predicted_matrix_impr[row['movieId']][row['userId']])
    return actual_results,predicted_results

In [21]:
MF_actual_ratings_imp,MF_predicted_ratings_imp = predictions_imp(testset)

print(' Matrix Factorization with regularisation term included to handle overfitting')
print("=================================================================================")
print('The Mean Absolute Error for test data is :', calculate_MAE(MF_actual_ratings_imp,MF_predicted_ratings_imp))
print('The Root Mean Squared Error for test data is :', calculate_RMSE(MF_actual_ratings_imp,MF_predicted_ratings_imp))


 Matrix Factorization with regularisation term included to handle overfitting
The Mean Absolute Error for test data is : 0.6804033486887275
The Root Mean Squared Error for test data is : 0.887131899302025


Please explain what you do to improve the recommendation in 1-2 paragraphs.

**A common extension to this basic MF algorithm is to introduce regularization to avoid overfitting. This is done by adding a parameter beta and modifing the squared error as implemented in the code above.We can observe the RMSE and MAE values have reduced for the MF implementation including regularisation factor thus indicating improvement in the basic algorithm. Regularization helps to choose preferred model complexity, so that model is better at predicting. Regularization is nothing but adding a penalty term to the objective function and control the model complexity using that penalty term. It attempts to reduce the variance of the estimator by simplifying it, something that will increase the bias, in such a way that the expected error decreases.In other words, the new parameter beta introduced is used to control the magnitudes of the user-feature and playlist-feature vectors such that P (Latent vector) and Q (Latent vector) would give a good approximation of R (Rating matrix) without having to contain large numbers. **

# Part 2. Bayesian Personalized Ranking (BPR) for Top-K Item Recommendation (30 points)

Compared with rating prediction in part 1, a more popular scenario recently is personalized top-K item ranking for each user based on the user's implicit feedback. Examples include ranking videos on YouTube and ranking products on Aamzon. In practice, users tend to provide implicit feedback (e.g., the user clicked a product URL on Amazon or played a video on YouTube) rather than explicit feedback (e.g., ratings or reviews) in most cases.

In this part, you will experiment with Bayesian Personalized Ranking (BPR) to rank items on a [Spotify Playlist Recommendation Dataset](http://people.tamu.edu/~yunhe/pubs/AttListCIKM2019.pdf). If a user ever followed a playlist, this interaction is treated as an implicit feedback. In our sampled dataset, there are ~10,000 users and ~7,000 playlists.

BPR can generate scores of items for each user. You should rank all items based on the scores for each user and evaluate the ranking performance.

For example, if user 0 has two interacted playlists 23, 78 in test.txt. If the top-10 playlists for user 0 returned by BPR is [12,45,78,34,23,90,134,33,46,9], then the precision@10 for user 0 is 0.2 because the two playlists in test.txt are recommended in top-10: 2/10=0.2. Please report NDCG@10 in this part.

## Load the Data

Please download the dataset from Piazza. There are about 90,000 interactions in total, which are split into training.txt, validation.txt and text.txt. You will train on train.txt, tune hyperparameters on validation.txt and report final result on test.txt in terms of NDCG@10. 

Each of the train and test files has lines having this format: UserID, PlaylistID, 1.0. 

First you will need to load the data and store it with any structure you like. Please report the numbers of unique users and movies in the dataset. 

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import math
from functools import reduce
from scipy.sparse import coo_matrix
import surprise
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
from collections import defaultdict
import math
from statistics import mean 
from lightfm import LightFM

C:\Users\eunna\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [23]:
#define file names

#Train files
train_txt = "train.txt"
train_csv = "trainc.csv"

#Test Files
test_txt = "test.txt"
test_csv = "testc.csv"


#Validation Files
vali_txt = "validation.txt"
vali_csv = "validationc.csv"


In [24]:
def parse_data(inputf,outputf):
    
    rows=['userId','PlaylistId',1] 
    
    with open(outputf, 'w+') as csvFile:
        writer = csv.writer(csvFile) 
        writer.writerow(rows)
        
        for f in open(inputf,"r"): 
            rowval=[]
            values=f.split()
            r=0;
            
            for i in range(len(rows)): 
                
                rowval.append(values[r])
                r += 1 
                
           
            writer.writerow(rowval)
            
    csvFile.close()
  

In [25]:
import csv
parse_data(train_txt,train_csv)
parse_data(test_txt,test_csv)
parse_data(vali_txt,vali_csv)

In [26]:
# load the data, then print out the number of
# playlists and users in each of train and test sets.
# Your Code Here...
train_data = pd.read_csv(train_csv)
print("         TRAIN DATA ")
print("==========================")
print(train_data.head())
print()
print("         TEST DATA ")
print("==========================")
test_data = pd.read_csv(test_csv)
print(test_data.head())
print()
print("     VALIDATION DATA ")
print("==========================")
vali_data = pd.read_csv(vali_csv)
print(vali_data.head())

unique_users_train_data =  len(train_data.userId.unique())
unique_playlist_train_data = len(train_data.PlaylistId.unique())

unique_users_test_data =  len(test_data.userId.unique())
unique_playlist_test_data = len(test_data.PlaylistId.unique())

unique_users_vali_data =  len(vali_data.userId.unique())
unique_playlist_vali_data = len(vali_data.PlaylistId.unique())

print()
print('Number of unique users in Train Data:', unique_users_train_data )
print('Number of unique playlist in Train Data:', unique_playlist_train_data )
print()
print('Number of unique users in Test Data :', unique_users_test_data )
print('Number of unique playlist in Test Data :', unique_playlist_test_data)
print()
print('Number of unique users in Validation Data :', unique_users_vali_data )
print('Number of unique playlist in Validation Data :', unique_playlist_vali_data)

         TRAIN DATA 
   userId  PlaylistId    1
0    3480         194  1.0
1    3480         657  1.0
2    3480         190  1.0
3    3480         660  1.0
4    3480        1437  1.0

         TEST DATA 
   userId  PlaylistId    1
0    5988         189  1.0
1    5988         105  1.0
2    5988        1400  1.0
3    5982        3589  1.0
4    5982        4259  1.0

     VALIDATION DATA 
   userId  PlaylistId    1
0    5988         124  1.0
1    5988        1878  1.0
2    5989        3418  1.0
3    5983        1379  1.0
4    5981        1037  1.0

Number of unique users in Train Data: 10183
Number of unique playlist in Train Data: 7787

Number of unique users in Test Data : 5846
Number of unique playlist in Test Data : 3604

Number of unique users in Validation Data : 5895
Number of unique playlist in Validation Data : 3674


## BPR by Using Package

Compared with MF, BPR is more complicated to implement. In this part, you can use a BPR package to experiment with top-K item recommendation. Some good packages include https://github.com/benfred/implicit.

In [27]:
# your code to call other BPR packages for top-K recommendation.
# Report average NDCG@10 for all users on test.txt


In [64]:
def FetchData(filename):
    f = open(filename, encoding = "utf8")      
    return f

In [65]:
def fetch_attributes(filename):
    f = FetchData(filename)
    uid = []
    pid = []
    rid = []
    for line in f:
        users = line.split('\t')[0]
        playlists = line.split('\t')[1]
        ratings = line.split('\t')[2]
        uid.append(users)
        pid.append(playlists)
        rid.append(ratings)
    return uid,pid,rid    

In [66]:
from scipy.sparse import coo_matrix
def form_coo_matrix(data,rows,cols):
    matrix = coo_matrix((data, (rows, cols)))
    return matrix

In [67]:
def NDCG(prob_rel,orginal_rel):
    count = 0
    DCG = 0
    IDCG = 0  
    iteration_len =len(prob_rel)
    for i in range(iteration_len):
        DCG += ((2**prob_rel[i])-1)/(math.log(1+i+1))
        IDCG += ((2**orginal_rel[i])-1)/(math.log(1+i+1))    
    if IDCG == 0:
        NDCG = 0
    else:
        NDCG = (DCG/IDCG)
    return NDCG

In [68]:
def bpr_LighFM(matrix_val,playlists,users_list,model):
    
    scores = []
    cal_ndcg = []
    ndcg = 0
    for uid in users_list:
        playlists_ratings = matrix_val.tocsr()[uid].indices  
        actual_list = [0,0,0,0,0,0,0,0,0,0]
        index = 0
        min_len = min(10, len(playlists_ratings))
        for i in range (1, min_len + 1):
            actual_list[index] = matrix_val.tocsr()[uid, playlists_ratings[i-1]]
            index = index +1    
        scores = model.predict(uid,np.arange(len(playlists)))
        top_items = np.argsort(-scores)
        predicted_list = []
        for i in range (1, 11):
            if top_items[i-1] in playlists_ratings:
                predicted_list.append(1)
            else:
                predicted_list.append(0)
       
        cal_ndcg.append(NDCG(predicted_list,actual_list))
    output = mean(cal_ndcg)
              
    return output     

In [97]:
#Tuned parameters
NUM_THREADS = 2
NUM_COMPONENTS = 50
USER_ALPHA = 5e-10
ITEM_ALPHA = 1e-4
LEARNING_RATE = 0.09
LEARNING_SCHEDULE = 'adadelta'
RANDOM_SEED = 29031994  
MAX_SAMPLED = 100
RHO =0.1
K_M = 50
N_M = 1
EPSILON = 1e-6

In [70]:
uid_train, pid_train, rid_train = fetch_attributes("train.txt")
uid_list = np.array(uid_train).astype(np.int)
pid_list = np.array(pid_train).astype(np.int)
rid_list = np.array(rid_train).astype(np.float)
unique_user_ids = list(dict.fromkeys(uid_list))


In [71]:
matrix = form_coo_matrix(rid_list,uid_list,pid_list)

In [72]:
uid_vali, pid_vali, rid_vali = fetch_attributes("validation.txt")
vuid = np.array(uid_vali).astype(np.int)
vpid = np.array(pid_vali).astype(np.int)
vrid = np.array(rid_vali).astype(np.float)
playlists_vali = list(dict.fromkeys(vpid))       #vali DATA
users_list_vali = list(dict.fromkeys(vuid))      #vali DATA

In [73]:
matrix_vali = form_coo_matrix(vrid,vuid,vpid)

In [96]:
#Model to tune best parameters to get maximum NDCG
def model_vali():
    model = LightFM(loss = 'bpr',learning_rate=LEARNING_RATE,
                    learning_schedule=LEARNING_SCHEDULE,
                    item_alpha=ITEM_ALPHA,
                    user_alpha=USER_ALPHA,
                    no_components=NUM_COMPONENTS,
                    max_sampled =MAX_SAMPLED,
                    rho = RHO,
                    k=K_M,
                    n=N_M,
                    epsilon = EPSILON,
                    random_state=RANDOM_SEED )
    ndcgg = []
    ndict = defaultdict(list)
    for k in range (100,200, 20):
        model.fit(matrix,epochs = k, num_threads = NUM_THREADS) 
        ngvali = bpr_LighFM(matrix_vali,playlists_vali,users_list_vali,model)
        ndict[ngvali] = k
        ndcgg.append(ngvali)
    epoch_value = ndict.get(max(ndcgg))    
    
    return epoch_value, max(ndcgg)

In [88]:
Max_epoch_value, max_ndcg_value  = model_vali()

In [89]:
print("Epoch value observed for maximum NDCG on Validation Dataset: ",Max_epoch_value)
print("NDCG@10 on Validation Dataset: ",max_ndcg_value)

Epoch value observed for maximum NDCG on Validation Dataset:  180
NDCG@10 on Validation Dataset:  0.08881777085547068


In [90]:
uid_test, pid_test, rid_test = fetch_attributes("test.txt")
tuid = np.array(uid_test).astype(np.int)
tpid = np.array(pid_test).astype(np.int)
trid = np.array(rid_test).astype(np.float)
playlists = list(dict.fromkeys(tpid))       #TEST DATA
users_list = list(dict.fromkeys(tuid))      #TEST DATA


In [91]:
matrix_test = form_coo_matrix(trid,tuid,tpid)   #test data matrix

In [92]:
def model_test(Max_epoch_value):
    model = LightFM(loss = 'bpr',learning_rate=LEARNING_RATE,
                    learning_schedule=LEARNING_SCHEDULE,
                    item_alpha=ITEM_ALPHA,
                    user_alpha=USER_ALPHA,
                    no_components=NUM_COMPONENTS,
                    max_sampled =MAX_SAMPLED,
                    rho = RHO,
                    k=K_M,
                    n=N_M,
                    epsilon = EPSILON,
                    random_state=RANDOM_SEED )
    
    model.fit(matrix,epochs = Max_epoch_value, num_threads = 2) 
    val = bpr_LighFM(matrix_test,playlists,users_list,model)
     
    
    return val

In [93]:
ndcg_final = model_test(Max_epoch_value)

In [95]:
#NDCG@10 on Test data using epoch value observed for maximum NDCG on Validation Data
print("NDCG@10 on Test Dataset: ",ndcg_final)

NDCG@10 on Test Dataset:  0.0906686756712301


## Collaboration declarations

*If you collaborated with anyone (see Collaboration policy at the top of this homework), you can put your collaboration declarations here.*

Part 1.A - Did Myself
Part 1.B - Referred approach from http://www.quuxlabs.com/blog/2010/09/matrix-factorization-a-simple-tutorial-and-implementation-in-python/
Part 1. C Did Myself
Part2 - Discussed with a classmate on the approach to be implemented.Did myself